In [1]:
import os, warnings
import wandb

import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedKFold

import params
warnings.filterwarnings('ignore')

In [2]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split")

wandb: Currently logged in as: gsparsh. Use `wandb login --relogin` to force relogin


In [27]:
raw_data_at = run.use_artifact(f'{params.CLEAN_DATA_AT}:latest')
path = Path(raw_data_at.download())

wandb: Downloading large artifact bcc_bigcats_clean:latest, 51.28MB. 591 files... 
wandb:   591 of 591 files downloaded.  
Done. 0:0:0.6


In [59]:
path.ls()

(#3) [Path('artifacts/bcc_bigcats_clean:v0/eda_table.table.json'),Path('artifacts/bcc_bigcats_clean:v0/bcc_images'),Path('artifacts/bcc_bigcats_clean:v0/media')]

In [60]:
fnames = orig_eda_table.get_column("File_Name")
groups = [s.split('-')[0] for s in fnames]

In [61]:
orig_eda_table = raw_data_at.get("eda_table")

wandb: Downloading large artifact bcc_bigcats_clean:latest, 51.28MB. 591 files... 
wandb:   591 of 591 files downloaded.  
Done. 0:0:0.1


In [62]:
y = orig_eda_table.get_column('Labels')
# y = [int(label=='Black Panther') for label in y] #Stratify by Black Panther Class

In [63]:
df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1

In [64]:
df.head()
len(df)

292

In [65]:
cv = StratifiedKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y)):
    df.loc[test_idxs, ['fold']] = i

In [68]:
df['stage'] = 'train'
df.loc[df.fold == 0, ['stage']] = 'test'
df.loc[df.fold == 1, ['stage']] = 'valid'
del df['fold']
df.stage.value_counts()

train    232
test      30
valid     30
Name: stage, dtype: int64

In [70]:
df.to_csv('data_split.csv', index=False)

In [71]:
processed_data_at = wandb.Artifact(params.PROCESSED_DATA_AT, type='split_data')

In [72]:
processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)

wandb: Adding directory to artifact (./artifacts/bcc_bigcats_clean:v0)... Done. 0.5s


In [73]:
data_split_table = wandb.Table(dataframe=df[['File_Name', 'stage']])

In [74]:
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "File_Name")

In [75]:
processed_data_at.add(join_table, 'eda_table_data_split')

ArtifactManifestEntry(path='eda_table_data_split.joined-table.json', digest='K5eiGyTK9Fww0c+ERiTPJA==', ref=None, birth_artifact_id=None, size=127, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmpym0r4z1n')

In [76]:
run.log_artifact(processed_data_at)
run.finish()

In [26]:
# import shutil

# path = 'artifacts';
# # Remove all directory content
# try:
#     shutil.rmtree(path)
# except:
#     print('Error deleting directory')